<a href="https://colab.research.google.com/github/rgukhui/cm4709/blob/main/YOLOv8_custom_dataset_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Training YOLOv8 on a Custom Dataset

By default, you can find weights of YOLOv8 trained on the COCO dataset which includes 80 classes of common objects in daily life.
If you have a specialised domain, you may want to train YOLO on a custom dataset.

##Aims
1. To create a custom dataset from videos using Roboflow.
1. To annotate a custom dataset using Roboflow.
1. To train YOLOv8 with a custom dataset.
1. Using a trained YOLOv8 network for object detection.

#Connect to Google Drive

As usual, we don't want to save files in the Colab runtime as they will get deleted when the runtime is disconnected. e.g. weights of the neural network after training.
It is recommended to save files our GoogleDrive space and mount it to your runtime.

In [ ]:
#mount Google drive to /content/drive
#
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Using Roboflow for Data Annotation

Roboflow is a free image annotation tool. Before using it, please be aware of the followings:
1. By default, datasets uploaded to Roboflow will be made public. **If your data are confidential, do not use Roboflow**.
1. With a free account, there is a limit on the number of images in the dataset.

##Creating a Roboflow Project
1. Log into your Google account as usual.
1. Go to [www.roboflow.com](http://www.roboflow.com). If it is the first time you use Roboflow, you can link it to your Google Account. Alternatively you can create a separate Roboflow account.
1. After signing into Roboflow, "Create a New Project".
1. Give the project a name. For this demo, we will create an "Object Detection" dataset.

##Creating Images from Video
While you can upload individual images to Roboflow, an easier way is to upload a video and ask Roboflow to sample frames from the video.

1. Drag-and-drop a video into your Roboflow project.
1. Choose the sample/frame-rate. Roboflow will tell you how many images will be sampled from the video. Usually you choose the frame rate depending on the nature of the video. e.g. If it contains fast-moving objects, you may want to use a higher frame rate.
1. Click the "Choose Frame Rate" button for Roboflow to complete the sampling and upload all images to your project.

##Assigning Annotation Task to People
Image annotation is usually the most time-consuming task as it require manual labelling (and also bounding box drawing in Object Detection). Roboflow allows you to assign annotation of an image subset to different people.

Roboflow also has an "Auto labelling" function, but for this simple demo we will do "Manual Labelling".

1. Click "Manual Labelling".
1. Assign all images to yourself for annotation.
1. Click "Assign Images" to complete the assignment.

##Annotation of Images
This is the most time-consuming task: For every image, you need to draw the bounding box of all "objects of interest" and label them with classes.

1. Click the "Start Annotating" button to start.
1. For each image: If there is no object, mark it as "Null". If there are objects, draw the bounding box and give the object a class label.

##Generate a Dataset
You can create different versions of datasets from the same set of images. e.g. Image sizes can be different, and you can apply pro-processing and augmentation.

1. Click "Generate" on the sidebar.
1. Add any pre-processing you want. e.g. Resize the image to 416x416 for YOLOv8, or convert images to greyscale if colour information is not important.
1. Optionally, add "Augmentation" to the images to cover more situations. e.g. flipping images horizontal. You may not want to flip vertically as in some domains, objects may never appear up-side-down. (Note that augmentation creates more images, which may exceed the image count in a free project.)

##Exporting a Dataset
Now your dataset is ready, you can export it to be used in Colab. We will come back to this later in Colab.


#Installing the YOLOv8 Library

To use YOLOv8 in Colab, we need to install the libraries from Ultralytics:

In [ ]:
%pip install ultralytics

import ultralytics
ultralytics.checks()

#Download Dataset from Roboflow

To download/import a dataset from Roboflow:
1. Go to Roboflow, your project, dataset, and click "Versions" on the side-bar.
1. Click "Export Dataset" button.
1. Select "YOLO V8" format, "show download code", and "Continue".
1. Copy the code and paste it into the code cell below.

Note: The generate code contains your Roboflow API key. **DO NOT disclose your API key to others.**

If you only want to try training YOLO but do not bother spending the time to annotate a dataset, you can use my [bird camera dataset](https://universe.roboflow.com/bird-hiao0/bird-camera).

In [ ]:
#
#make folder to store dataset
#and change directory to there before downloading
#
%mkdir /content/datasets
%cd /content/datasets

#
#the follow code is copied from Roboflow datset export
#Note: It contains an API key which you MUST NOT show to others.
#

!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="***Roboflow API key here***")
project = rf.workspace("bird-hiao0").project("bird-camera")
version = project.version(8)
dataset = version.download("yolov8")


#Fixing the `data.yaml` File

The downloaded dataset is in the `/content/datasets` folder of your Colab runtime.

The `data.yaml` file specifies parameters of the dataset. Unfortunately, the generated `data.yaml` is not quite correct. In particular, the paths to the training and validation subsets are wrong. We need to fix 2 errors before we can use the dataset:

1. In Colab, look for the `data.yaml` file under the `datasets/xxxx-nn` folder, where `xxxx` is your Roboflow project name, and `nn` is the dataset version number.
1. Look for 2 lines that look similar to this:

```
train: xxxx-nn/train/images
val: xxxx-nn/valid/images
```

Change them to the followings:

```
train: ../train/images
val: ../valid/images
```


#Training YOLO V8

With the YOLO V8 library and the dataset ready, we can now train our model. You may want to modify the following parameters:
* `model`: This is the YOLO model to use. This defines the size and scale of the network.
* `epochs`: The number of epochs to train.
* `imgsz`: Input image size to YOLO. V8 default size is 640.

See the Ultralytics documentation [here](https://docs.ultralytics.com/modes/train/#key-features-of-train-mode) on how to train and the meaning of these parameters.

In [ ]:
#
#train YOLO using yolov8n model, 50 epoch, image size 416
#
# Note: Modify "data" value to point to "data.yaml" in dataset.
#
!yolo task=detect mode=train model=yolov8n.pt data=/content/datasets/bird-camera-8/data.yaml epochs=50 imgsz=416 plots=True

#
#Optionally, copy best weights into Google Drive space after training
#
#Note: Weight files are put into subfolder "train", "train2", etc. in subsequent training runs.
#
!mkdir /content/drive/MyDrive/YOLO/v8/weights
!cp /content/runs/detect/train/weights/best.pt /content/drive/MyDrive/YOLO/v8/weights/yolov8n.pt

#Object Detection from Command Line

To do object detection in Command Line, we only need the paths to the weights file and the image.


In [ ]:
!yolo detect predict model=/content/runs/detect/train/weights/best.pt source='/content/drive/MyDrive/cm4709/YOLOv8 training demo/images/robin1.png'

#Object Detection Using PyTorch

The following codes ar mostly from Lab06 in using YOLO v8 in PyTorch:

In [ ]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
from ultralytics import YOLO

#our testing images

image1='/content/drive/MyDrive/cm4709/YOLOv8 training demo/images/bluetit1.png'
image2='/content/drive/MyDrive/cm4709/YOLOv8 training demo/images/bluetit2.png'
image3='/content/drive/MyDrive/cm4709/YOLOv8 training demo/images/robin1.png'
image4='/content/drive/MyDrive/cm4709/YOLOv8 training demo/images/robin2.png'
images=[image1,image2,image3,image4]

#weights file path
weightsFile='/content/runs/detect/train/weights/best.pt'

model = YOLO(weightsFile)  # load a pretrained YOLOv8s detection model
results=model.predict(images,agnostic_nms=True,iou=0.6)  # predict on images
result=results[0]     #get 1st result
classNames=result.names #get class names

print('No. of classes: ',len(classNames))
print(classNames)

In [ ]:
#
#same code as above to process results
#

#lists to collect results
originalImages=[]
detectionResults=[]

for result in results:
  img=result.orig_img       #get original image
  originalImages.append(img)  #append image to list
  boxResults=[]
  for box in result.boxes:  #go through all boxes
    #
    #compute values from tensors
    #
    [classIndex]=box.cls.cpu().numpy()    #get class index of box
    classIndex=int(classIndex)            #convert to int
    label=classNames[classIndex]          #get class name as label
    [confidence]=box.conf.cpu().numpy()   #get box confidence
    label=label+' '+str(confidence)       #append confidence to label
    [[x,y,w,h]]=box.xywh.cpu().numpy()    #get box centre, width and height
    topx=int(x-w/2)       #compute box top left corner
    topy=int(y-h/2)
    bottomx=int(topx+w)   #compute box bottom right corner
    bottomy=int(topy+h)
    #
    #append box result to list
    #
    boxResults.append((classIndex,confidence,label,(topx,topy,bottomx,bottomy)))
  detectionResults.append(boxResults)

#
#same code as above to show images and boxes
#

#font to be used in label
font = cv.FONT_HERSHEY_SIMPLEX

#generate random colours for the classes
colours = np.random.uniform(0, 255, size=(len(classNames), 3))

#
#go through all images
#
for index in range(0,len(originalImages)):
  image=originalImages[index]         #take out the image
  boxResults=detectionResults[index]  #get all box results of this image
  for boxResult in boxResults:        #go through all boxes in this result
    (classIndex,confidence,label,(topx,topy,bottomx,bottomy))=boxResult
    colour = colours[classIndex]                 #get colour
    cv.rectangle(image, (topx,topy), (bottomx,bottomy), colour, 2)  #draw bounding box
    cv.putText(image, label, (topx,topy -10), font, 0.5, colour)    #draw class label
  #
  #show image
  #
  plt.figure(figsize=(10,10))
  plt.imshow(cv.cvtColor(image, cv.COLOR_BGR2RGB))

# Export Weights to ONXX

After training, the "weights" file is in PyTorch format. To use it in other platforms (e.g. OpenCV), you can export the `.pt` weight file into `.onnx` ([Open Neural Network Exchange](https://onnx.ai/)) format.

In [ ]:
#
# Change the weight file path accordingly.
#
#Note: OpenCV requires the use of "opset=12" and "simplify" for compatibility.
#
!yolo task=detect mode=export model=/content/runs/detect/train/weights/best.pt format=onnx opset=12 simplify imgsz=416